### Load documents list

In [1]:
import os

directory_path = 'data/'
document_list = os.listdir(directory_path)
print(document_list)

['Attention-is-All-You-Need.pdf', 'GPT-4-Technical-Report.pdf', 'Lets-Verify-Step-by-Step.pdf', 'Sparks-of-AGI.pdf', 'STaR-Self-Taught-Reasoner.pdf', 'Tree-of-Thoughts.pdf']


### Parse documents into lists of text and create its metadata

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pypdf import PdfReader
import uuid

documents = []
metadatas = []
ids = []

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 100,
    length_function = len,
    is_separator_regex = False,
)

for document_name in document_list:
    reader = PdfReader(directory_path + document_name)
    for page in reader.pages:
        page_text = page.extract_text()
        chunks = text_splitter.create_documents([page_text])
        for chunk in chunks:
            documents.append(chunk.page_content)
            metadatas.append({'source': document_name, 'page_number': reader.get_page_number(page)})
            ids.append(str(uuid.uuid4()))

### Create DB connection and embeddings function
Here we are using in memory data base and free model from: https://www.sbert.net/docs/pretrained_models.html

In [3]:
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.EphemeralClient()
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

### Create and store documents data as embeddings in vector data base

In [4]:
collection = chroma_client.create_collection(name="pdf_data", embedding_function=sentence_transformer_ef)
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

### Get user querry, find related information in DB and append it to querry

In [5]:
user_querry = "Hi there! I would like to know why self-attention is so important?"

### Connect to LLM and extract key information from user prompt
This approach gives much better results for vector DB search. 

Extraction of key information can be performed by smaller LLM for cost efficiency.

In [6]:
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  temperature=0.0,
  messages=[
    {"role": "system", "content": "You will convert user question to string suitable for data base search. Have in mind that we need only essence for search."},
    {"role": "user", "content": user_querry}
  ]
)

user_querry_sumerized = completion.choices[0].message.content
print(user_querry_sumerized)

"importance of self-attention"


### Find revelant .pdf files that might consist answer for the user

In [7]:
results = collection.query(
    query_texts=[user_querry_sumerized],
    n_results=10
)

# Context can be improved by providing adjacent chunks from the same document based on metadata
# In this proof of concept approach of simply combining most relevant chunks of information is good enough
context = results["documents"]

### Prepare system message for LLM

In [8]:
# Basic system message that should be improved by taking into account fragmented context from different files.
# Furthermore we can use here metadatas and give LLM instruction to backup its answer with references to documents.
system_message = """\
Context information is below.
---------------------
{context}
---------------------
Given the context information and not prior knowledge answer user question.
If context does not contain answer, tell the user you didin't find answer.
"""

### Generate answer for user question with high quality LLM and provided context

In [9]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  temperature=0.0,
  messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_querry}
  ]
)

answer = completion.choices[0].message.content
print(answer)

Self-attention is important because it allows a model to weigh the importance of different words in a sequence when making predictions. This helps the model capture long-range dependencies and understand the relationships between words in the input sequence. Self-attention has been widely used in natural language processing tasks such as machine translation and text generation, and has been shown to improve the performance of these models.
